In [ ]:
import sys

message = 'Hello'
items = tuple(["Name","Ishe Doro","Course","Python Security","Hours",10])

print("Size of message: ", sys.getsizeof(message))
print("Size of message: ", sys.getsizeof(items))

In [ ]:
print("Size of message hash: ", sys.getsizeof(hash(message)))
print("Size of list hash: ", sys.getsizeof(hash(items)))

In [ ]:
class MyData:

    def __hash__(self):
        # Implement hash value generation

In [ ]:
# AVALANCHE EFFECT
print(abs(int(hex(0X02),16) - int(hex(0X03),16))) # Small diff
print(abs(hash(hex(0X02)) - hash(hex(0X03)))) # Large diff in hash value

In [ ]:
class MyItem:

    def __init__(self,name):
        self.name = name
    
    def __hash__(self):
        return hash(self.name)

In [ ]:
item = MyItem("Ishe")
item2 = MyItem("Ishe Doro")
hash(item) == hash(item2)

In [ ]:
print(hash(-1))
print(hash(-2))

In [ ]:
# HASHLIB

#Named constructor
import hashlib
import json
from hashlib import sha256

data = str.encode(json.dumps([{"Val":x, "Val Squared":x**2} for x in range(10)]))
sha256_obj = sha256()
sha256_obj_1 = sha256(bytes(data))

#Generic constructor
# Secure context
sha256_obj_generic = hashlib.new("sha256",bytes(data)) 
# Non-secure
sha256_obj_generic_nonsecure = hashlib.new("sha256",usedforsecurity=False) 

In [ ]:
# Secure Hash As Bytes
sha256_obj_1.digest()

In [ ]:
# Secure Hash As Hexadecimal
sha256_obj_1.hexdigest()

In [ ]:
hash("Hello World")

In [ ]:
message ="Hello World"
sha256_obj.update(str.encode(message))
sha256_obj.hexdigest()

In [ ]:
message ="Hello World"
sha256_obj.update(str.encode(message)) #update is called incrementally
sha256_obj.hexdigest() # Recomputes the secure hash for the concantenated message

In [ ]:
sha256(str.encode(message)).hexdigest()

In [ ]:
# Incremental use of update
message_list =["Hello"," ","World"]
sha256_obj_a = sha256()
for m in message_list:
    sha256_obj_a.update(str.encode(m))
sha256_obj.hexdigest()

In [ ]:
from hmac import compare_digest
def generate(data):
    # Incremental use of update
    sha256_obj = sha256(str.encode(data))
    return {"Data":data, "Hash":sha256_obj.hexdigest()}

def verify(message):
    data, secure_hash = message
    sha256_obj = sha256(message["Data"])
    if compare_digest(sha256_obj.hexdigest(),secure_hash):
        print("Message okay")
    else:
        print("Message corrupted")

In [ ]:
message = generate("This is a test message")
verify(json.dumps(message))

In [ ]:
hashlib.algorithms_guaranteed

In [ ]:
len(hashlib.algorithms_available)

In [ ]:
'AAAAAAAAAAAAAB7849' # Hexadecimal 16 possible 0 - 9 A - F
'AA0000000000000000' # N bytes long

In [ ]:
import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

key = secrets.token_bytes(32)

cipher = Cipher(algorithms.AES(key),modes.ECB())

# encryption CipherContext instance
def encrypt_ecb(data):
    encryptor = cipher.encryptor()
    return encryptor.update(data) + encryptor.finalize()

# decrypt CipherContext instance
def encrypt_ecb(data):
    decryptor = cipher.decryptor()
    return decryptor.update(data) + decryptor.finalize()

In [ ]:
import os
# Cipher Block Chaining Mode
def encrypt(data):
    iv = secrets.token_bytes(16)
    cipher = Cipher(algorithms.AES(key),modes.CBC(iv))
    encryptor = cipher.encryptor()
    return (encryptor.update(data) + encryptor.finalize(),iv)

def decrypt(ciphertext):
    ct,iv = ciphertext
    cipher = Cipher(algorithms.AES(key),modes.CBC(iv))
    decryptor = cipher.decryptor()
    return (decryptor.update(ct) + decryptor.finalize(),iv)

In [ ]:
from cryptography.hazmat.primitives import padding

padder = padding.PKCS7(128).padder()
message = padder.update(b"Hello there") + padder.finalize()
ciphertext_1 = encrypt(message)
ciphertext_2 = encrypt(message)
print(ciphertext_1)
print(ciphertext_2)

In [ ]:
message

In [ ]:
# Galois Counter Mode (GCM) - Emulate a stream cipher

def authenticated_encrypt_gcm(plaintext,associated_data):
    iv = secrets.token_bytes(10)
    encryptor = Cipher(algorithms.AES(key),modes.GCM(iv)).encryptor()
    encryptor.authenticate_additional_data(associated_data)
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    return (iv, ciphertext,encryptor.tag)

def decrypt_with_gcm(token,associated_data):
    iv, ciphertext,tag = token
    decryptor = Cipher(algorithms.AES(key),modes.GCM(iv,tag)).decryptor()
    decryptor.authenticate_additional_data(associated_data)
    return decryptor.update(ciphertext) + decryptor.finalize()
    

In [ ]:
ciphertext,iv = encrypt(message)

In [ ]:
cipher_file = open('cipher_file.tk','w')
cipher_file.writelines(ciphertext.hex() + " - " + iv.hex())
cipher_file.close()

In [ ]:
cipher_file = open('cipher_file.tk','r')
token = cipher_file.readline()
cipher_file.close()

In [ ]:
token.split(' - ')

In [ ]:
# RSA KEY GENERATION
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(public_exponent=65537,key_size=3072)
public_key = private_key.public_key()

In [ ]:
private_bytes = private_key.private_bytes(encoding=serialization.Encoding.PEM,
format=serialization.PrivateFormat.PKCS8,encryption_algorithm=serialization.NoEncryption())

with open('my_private_key.pem','wb') as private_key_file:
    private_key_file.write(private_bytes)




In [ ]:
public_bytes = public_key.public_bytes(encoding=serialization.Encoding.PEM,
format=serialization.PublicFormat.SubjectPublicKeyInfo)

with open('public_key_file.pem','wb') as public_key_file:
    public_key_file.write(public_bytes)

In [3]:
from cryptography.hazmat.primitives import serialization

def load_key(is_private=True):
    if is_private:
        with open('my_private_key.pem','rb') as my_private_key:
            return serialization.load_pem_private_key(my_private_key.read(),password=None)
    else:
        with open('public_key_file.pem','rb') as public_key_file:
            return serialization.load_pem_public_key(public_key_file.read())

In [ ]:
# RSA Encryption
# OAEP - Optional asymmetric encryption padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

padding_config = padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None)
public_key_from_disk = load_key(False)
ciphertext = public_key_from_disk.encrypt(plaintext=b"Sample data",padding=padding_config)

In [ ]:
ciphertext

In [ ]:
# Decrypt with private key
load_key().decrypt(ciphertext=ciphertext,padding=padding_config)

In [14]:
# RSA Digital Signature
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
import json

message = bytes([x for x in range(1,6)])

padding_config = padding.PSS(mgf=padding.MGF1(algorithm=hashes.SHA256()),salt_length=padding.PSS.MAX_LENGTH)
signature = load_key().sign(message,padding_config,hashes.SHA256())
signed_message = {
    'message' : message,
    'signature' : signature
}

In [15]:
from cryptography.exceptions import InvalidSignature
def verify(data):
    message = data['message'] 
    signature = data['signature']
    public_key = load_key().public_key()

    try:
        public_key.verify(signature,message,padding_config,hashes.SHA256())
        print("Trusted message and source")
    except InvalidSignature:
        print("Untrustworthy source")


In [16]:
verify(signed_message)

Trusted message and source


In [ ]:
# ECDSA vs RSA
# private <--> public vice versa
# Very fast for signing data

# 256 bits vs 2048 bit RSA key

In [18]:
from cryptography.hazmat.primitives.asymmetric import ec

ec_private_key = ec.generate_private_key(ec.SECP384R1())
signature = ec_private_key.sign(b"Data",ec.ECDSA(hashes.SHA256()))

In [22]:
try:
    ec_private_key.public_key().verify(signature,b"Data M",ec.ECDSA(hashes.SHA256()))
except InvalidSignature:
    print("Untrustworthy data")

Untrustworthy data
